Tham so model

In [6]:
# Sets - dùng list comprehension để tạo các tập hợp số nguyên theo khoảng
V = [i for i in range(1, 10)] # Vessels - Cập nhật số lượng tàu thực tế
B = [j for j in range(1, 5)] # Berths - Cập nhật số lượng bến thực tế
C = [n for n in range(1, 8)] # Cranes - Cập nhật số lượng cần cẩu thực tế
T = [t for t in range(1, 48)] # Time slots - 48 khe thời gian (2 ngày)
Y = [y for y in range(1, 20)] # Trucks - Cập nhật số lượng xe tải thực tế

# Các tập đặc biệt
VL = [1, 3, 5] # Tập hợp các tàu lớn
VO = [k for k in range(1, len(V) + 1)] # Thứ tự vào cảng
VB = [j for j in range(1, len(B) + 1)] # Số sê-ri của bến 
VC = [n for n in range(1, len(C) + 1)] # Số sê-ri của cần cẩu
CK = [[0 for t in range(len(T))] for n in range(len(C))] # Số xe tải phục vụ mỗi cần cẩu n tại thời điểm t
VK = [3 for i in range(len(V))] # Số xe tải vận hành mỗi tàu

# Parameters
W = 2 # Độ dài cửa sổ thời gian thủy triều cao/thấp
R = 10000 # Số nguyên đủ lớn
M = 30 # Tổng số xe tải có sẵn
TO = [8, 10, 12, 15, 17, 20, 22, 25, 27] # Thời gian dự kiến đến của các tàu
TG0 = [2 for i in range(len(V))] # Thời gian di chuyển qua kênh
CE0 = 30 # Hiệu suất lý thuyết của cần cẩu bến cảng
CF0 = 15 # Hiệu suất lý thuyết của cần cẩu tại bãi container
TW0 = 6 # Thời gian chờ tối đa của tàu

# Ship parameters
VL_ships = [80, 100, 150, 120, 90, 110, 130, 140, 95] # Chiều dài của tàu i
VD_ships = [8, 10, 12, 9, 7, 11, 13, 14, 9] # Mớn nước của tàu i
VE_ships = [200, 300, 500, 400, 250, 350, 450, 550, 280] # Số container của tàu i

# Berth parameters
BL_berths = [200, 250, 180, 300] # Chiều dài của bến j
BD_berths = [15, 18, 14, 20] # Mớn nước của bến j

# Distance matrix D[i][j] - khoảng cách từ tàu i ở bến j đến bãi container
D = [
    [5, 8, 10, 12],
    [6, 7, 9, 11], 
    [7, 6, 8, 10],
    [8, 9, 7, 11],
    [9, 10, 11, 8],
    [10, 9, 12, 7],
    [11, 12, 10, 6],
    [12, 11, 9, 5],
    [10, 8, 7, 9]
]

# Giới hạn trên/dưới số cần cẩu phục vụ tàu
VCm = [1, 1, 2, 1, 1, 1, 2, 2, 1] # Giới hạn dưới
VCM = [3, 2, 4, 3, 2, 3, 4, 4, 2] # Giới hạn trên

# Dependent variables
TA = [] # Thời gian đến thực tế của tàu i
TB = [] # Thời gian bắt đầu vào kênh của tàu i
TC = [] # Thời điểm tàu đã vào cảng
TS = [] # Thời điểm bắt đầu xếp/dỡ hàng
TD = [] # Thời điểm kết thúc xếp/dỡ hàng
TV = [] # Thời điểm bắt đầu hỗ trợ từ cần cẩu lân cận
TE = [] # Thời điểm bắt đầu ra khỏi kênh
TF = [] # Thời điểm tàu rời cảng

# Thời gian vận chuyển
TH = [[0 for j in range(len(B))] for i in range(len(V))] # Thời gian vận chuyển không tải
TL = [[0 for j in range(len(B))] for i in range(len(V))] # Thời gian vận chuyển có tải

# Số lượng cần cẩu phục vụ tàu i tại thời điểm t
VCit = [[0 for t in range(len(T))] for i in range(len(V))]

# Số lượng xe tải phục vụ cần cẩu n tại thời điểm t
CKnt = [[0 for t in range(len(T))] for n in range(len(C))]

# Hiệu suất làm việc thực tế của cần cẩu
CE = [CE0 * 0.85 for n in range(len(C))]  # 85% hiệu suất lý thuyết

# Liên quan đến xe tải
Dl = 0  # Tổng quãng đường xe tải đi khi không tải
Dk = 0  # Tổng quãng đường xe tải đi khi có tải
tn_ = 0  # Tổng thời gian xe tải chạy không tải

# Tốc độ xe tải
vh = 40  # Tốc độ khi không tải (km/h)
vl = 25  # Tốc độ khi có tải (km/h)
E1 = 2.65 #Hệ số phát thải carbon của xe tải

# Biến hỗ trợ thủy triều
u_iu = [[0 for u in range(10)] for i in range(len(V))]  # Khung thời gian vào
v_iu = [[0 for u in range(10)] for i in range(len(V))]  # Khung thời gian ra

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Dữ liệu mẫu: [v^2, v, l, v*l, 1]
X = np.array([[1600, 40, 0, 0, 1], [3600, 60, 2, 120, 1]])
y = np.array([20.5, 25.8])

model = LinearRegression().fit(X, y)
a, b, c, d, e = model.coef_[0], model.coef_[1], model.coef_[2], model.coef_[3], model.intercept_
print(f"Hệ số: a={a:.2f}, b={b:.2f}, c={c:.2f}, d={d:.2f}, e={e:.2f}")

Hệ số: a=0.00, b=0.00, c=0.00, d=0.00, e=16.27


1. Hệ số mặc định từ nghiên cứu Barth et al. (2005)
Nếu bạn không có dữ liệu riêng và đang làm mô phỏng tổng quát, sử dụng các hệ số từ nghiên cứu được trích dẫn trong tài liệu của bạn ([46, 47]):

a=0.02,b=−1.67,c=0.46,d=0.03,e=51.17
Phù hợp cho: Xe tải hạng nặng (heavy-duty trucks), container.

Nguồn:

Barth, M., et al. (2005). Comprehensive Modal Emissions Model (CMEM).

2. Hệ số từ EPA (Environmental Protection Agency)
Nếu bạn cần hệ số tiêu chuẩn cho xe tải diesel (phổ biến trong các báo cáo môi trường):

a=0.025,b=−1.8,c=0.5,d=0.035,e=50.0
Ưu điểm: Được sử dụng rộng rãi trong tính toán phát thải.

Nguồn:

EPA. *Emission Factors for Heavy-Duty Vehicles* (2010).

In [7]:
def calculate_F1(TF, TA):
    v = len(V)
    total = sum(TFi - TAi for TFi, TAi in zip(TF, TA))
    F1 = (1 / v) * total
    return F1

def calculate_pvl(v, l):
    a = 0.02
    b = -1.67
    c = 0.46
    d = 0.03
    e = 51.17
    return a * v * v + b * v + c * l + d * v * l + e

def calculate_F2(E1, v, lmin, lmax, Dl, Dk, n_, tn_):
    pl = calculate_pvl(v, lmin)
    pk = calculate_pvl(v, lmax)
    F2 = E1 * (pl * Dl + pk * Dk + n_ * tn_)
    return F2
    
def calculate_F3(TD, TS, TC, TV, E2, delVCit, lambda1, lambda2):
    for i in range(0, len(TD)):
        part1 += (TD[i] - TS[i]) * VC[i]
        part2 += (TD[i] - TV[i]) * delVCit[i]
        part3 += delVCit[i]
    F3 = E2 * (lambda1 * (part1 + part2) + lambda2 * part3)
    return F3

def calculate_k123(a, b, c):
    return float(1)/a, float(1)/b, float(1)/c

F1 = calculate_F1(TF, TA)
F2 = calculate_F2(E1, v, 0, l, Dl, Dk, n_, tn_)
F3 = calculate_F3(TD, TS, VC, TV, E2, delVCit, lambda1, lambda2)
k1, k2, k3 = calculate_k123(F1, F2, F3)
w1 = 0.4
w2= 0.3
w3 = 0.3

F = min(w1 * k1 * F1 + w2 * k2 * F2 + w3 * k3 * F3)

NameError: name 'v' is not defined

In [ ]:
# from pulp import *

# # Tạo bài toán
# prob = LpProblem("TEU-BQCT_Optimization", LpMinimize)

# # Biến quyết định
# x = LpVariable.dicts("x", [(i,j) for i in range(v) for j in range(m)], cat='Binary')

# # Ràng buộc xe tải
# for i in range(v):
#     prob += lpSum(x[(i,j)] for j in range(m)) == 1  # Mỗi container chỉ gán 1 xe

# # Ràng buộc năng lực cần cẩu
# for t in time_slots:
#     prob += lpSum(TC[i][t] for i in range(v)) <= Q_max

# # Giải bài toán
# prob.solve()

NameError: name 'v' is not defined

In [ ]:
def calculate_TA(TO, delTAi):
    for i in range(0, len(TO)):
        TA.append(TO[i] + delTAi[i])
    return TA

def calculate_fErlang():
    for i in range(0, len(x)):
        fxku_ = (pow(u_TA * x[i], kTA[i] - 1) * pow(u_TA * e, -1 * u_TA * x[i]))
        delTA.append(fxku_)
    return delTA

calculate_fErlang() # (8)
TAi = calculate_TA(TOi, delTA) # (7)

In [ ]:
def constraints_9(tb, ta):
    count = 0
    for i in range(len(V)):
        if ta[i] > tb[i]:
            count += 1
            print(f"Constraints error (9). TB[{i+1}] need to greater than with TA[{i+1}]")
    if count == 0:
        print("Passed constraints (9)")

constraints_9(TB, TA)

In [ ]:
def constraints_10(tb, tg):
    for i in range(len(V)):
        TC.append(tb[i] + tg[i])
    return TC

constraints_10(TB, TG0)

In [ ]:
def constraints_11(ts, tc):
    count = 0
    for i in range(len(V)):
        if ts[i] > tc[i]:
            count += 1
            print(f"Constraints error (11). TS[{i+1}] need to greater than with TC[{i+1}]")
    if count == 0:
        print("Passed constraints (11)")

constraints_11(TS, TC)

In [ ]:
def constraints_12(ts, ta, tw):
    count = 0
    for i in range(len(V)):
        if (ts[i] - ta[i]) > tw[i]:
            count += 1
            print(f"Constraints error (12). TS[{i+1}] - TA[{i+1}] need to lower than or equal with TW[{i+1}]")
    if count == 0:
        print("Passed constraints (12)") 
        
constraints_12(TS, TA, TW)

In [ ]:
def constraints_13(td, ts, ve, ce, t , vcit):
    count = 0
    for i in range(len(V)):
        if (td[i] - ts[i]) == ve[i]/(ce * t[i] * vcit[i]):
            pass
        else:
            count += 1
            print(f"Constraints error with i = {i + 1}")
    if count == 0:
        print("Passed constraints (13)") 

constraints_13(TD, TS, VE, CE, T, VCit)

In [ ]:
def constraints_14(te, td):
    count = 0
    for i in range(len(V)):
        if te[i] > td[i]:
            count += 1
            print(f"Constraints error (14). TE[{i+1}] need to greater than with TD[{i+1}]")
    if count == 0:
        print("Passed constraints (14)")

constraints_14(TE, TD)

In [ ]:
def constraints_15(tf, te, tg):
    count = 0
    for i in range(len(V)):
        if tf[i] == te[i] + tg[i]:
            pass
        else:
            count +=1
            print(f"Constraints error with i = {i + 1}")
    if count == 0:
        print("Passed constraints (14)") 

constraints_15(TF, TE, TG0)

In [ ]:
def constraints_18_19():
    S18 = 0
    S19 = 0
    K18 = 0
    K19 = 0
    for i in range(len(Tt)):
        S18 += u_iu[i]
        if S18 == 1:
            K18 = i
    for i in range(len(Tt)):
        S19 += v_iu[i]
        if S19 == 1:
            K19 = i
    if S18 == 1:
        print(f"Pass constraints (18), tau duoc phep di trong khung thu {K18 + 1}")
    else:
        print("Contraints (18) error")
    if S19 == 1:
        print(f"Pass constraints (19), tau duoc phep di trong khung thu {K19 + 1}")
    else:
        print("Contraints (19) error")

In [ ]:
import numpy as np

def constraints_20(x, BD, VD):
    """
    Kiểm tra ràng buộc: 
    ∑_j ∑_k (x_ijk * BD_j) ≥ VD_i với ∀ i ∈ V

    Parameters:
        x (numpy.ndarray): Ma trận nhị phân (I x J x K) – x[i][j][k] = 1 nếu tàu i được phục vụ tại bến j theo thứ tự k.
        BD (numpy.ndarray): Vector chiều dài hoặc công suất của các bến (J,)
        VD (numpy.ndarray): Nhu cầu chiều dài hoặc công suất của các tàu (I,)

    Returns:
        bool: True nếu tất cả các ràng buộc đều thỏa mãn, False nếu có tàu nào đó vi phạm.
    """
    I, J, K = x.shape

    for i in range(I):
        total_capacity = 0
        for j in range(J):
            for k in range(K):
                total_capacity += x[i][j][k] * BD[j]
        if total_capacity < VD[i]:
            print(f"Ràng buộc không thỏa mãn cho tàu {i}: Tổng = {total_capacity} < Nhu cầu = {VD[i]}")
            return False
    return True


In [ ]:
# Giả sử có 2 tàu (I = 2), 3 bến (J = 3), 2 thứ tự (K = 2)
x = np.array([
    # Tàu 0
    [[1, 0], [0, 0], [0, 0]],
    # Tàu 1
    [[0, 0], [1, 0], [0, 1]]
])

BD = np.array([100, 150, 200])   # Năng lực các bến
VD = np.array([90, 300])         # Nhu cầu các tàu

result = constraints_20(x, BD, VD)
print("Ràng buộc thỏa mãn!" if result else "Có tàu vi phạm ràng buộc.")


In [ ]:
import numpy as np

def constraints_21(x, BL, VL):
    """
    Kiểm tra ràng buộc:
    ∑_j ∑_k (x_ijk * BL_j) ≥ VL_i với ∀ i ∈ V

    Parameters:
        x (numpy.ndarray): Ma trận nhị phân (I x J x K), x[i][j][k] = 1 nếu tàu i được phục vụ tại bến j theo thứ tự k.
        BL (numpy.ndarray): Vector chiều dài các bến (J,)
        VL (numpy.ndarray): Vector chiều dài yêu cầu của các tàu (I,)

    Returns:
        bool: True nếu tất cả tàu đều thỏa mãn ràng buộc chiều dài bến, False nếu có tàu không thỏa mãn.
    """
    I, J, K = x.shape

    for i in range(I):
        total_length = 0
        for j in range(J):
            for k in range(K):
                total_length += x[i][j][k] * BL[j]
        if total_length < VL[i]:
            print(f"Ràng buộc không thỏa mãn cho tàu {i}: Tổng chiều dài bến = {total_length} < Chiều dài tàu = {VL[i]}")
            return False
    return True


In [ ]:
# Giả sử có 2 tàu (I = 2), 3 bến (J = 3), 2 thứ tự (K = 2)
x = np.array([
    # Tàu 0
    [[1, 0], [0, 0], [0, 0]],
    # Tàu 1
    [[0, 0], [1, 0], [0, 1]]
])

BL = np.array([120, 180, 150])   # Chiều dài các bến
VL = np.array([100, 300])        # Yêu cầu chiều dài các tàu

result = constraints_21(x, BL, VL)
print("Ràng buộc chiều dài bến thỏa mãn!" if result else "Có tàu vi phạm ràng buộc chiều dài bến.")


In [ ]:
import numpy as np

def constraints_22_23(x):
    """
    Kiểm tra ràng buộc:
    (22) ∑_i x_ijk ≤ 1 ∀j, k
    (23) ∑_j ∑_k x_ijk = 1 ∀i

    Parameters:
        x (numpy.ndarray): Ma trận nhị phân (I x J x K), x[i][j][k] = 1 nếu tàu i được phục vụ tại bến j theo thứ tự k.

    Returns:
        bool: True nếu thỏa mãn cả hai ràng buộc, False nếu không.
    """
    I, J, K = x.shape
    
    # Ràng buộc (22)
    for j in range(J):
        for k in range(K):
            total = sum(x[i][j][k] for i in range(I))
            if total > 1:
                print(f"Vi phạm ràng buộc (22) tại bến j={j}, thứ tự k={k} (tổng = {total})")
                return False

    # Ràng buộc (23)
    for i in range(I):
        total = sum(x[i][j][k] for j in range(J) for k in range(K))
        if total != 1:
            print(f"Vi phạm ràng buộc (23) tại tàu i={i} (tổng = {total})")
            return False

    return True


In [ ]:
# 2 tàu, 2 bến, 2 thứ tự
x = np.array([
    # Tàu 0
    [[0, 1], [0, 0]],
    # Tàu 1
    [[1, 0], [0, 0]],
])

result = constraints_22_23(x)
print("Thỏa mãn ràng buộc (22) và (23)!" if result else "Có vi phạm!")


In [ ]:
import numpy as np

def constraint_24(x, TA):
    """
    Kiểm tra ràng buộc thứ tự thời gian phục vụ tàu:
    x_ijk * TA_i ≤ x_i'j(k+1) * TA_i', ∀i, i' ∈ V, ∀j ∈ B, ∀k ∈ VO

    Parameters:
        x (numpy.ndarray): Ma trận nhị phân (I x J x K), x[i][j][k] = 1 nếu tàu i được phục vụ tại bến j theo thứ tự k.
        TA (numpy.ndarray): Mảng thời gian đến của tàu i, có kích thước (I,).

    Returns:
        bool: True nếu thỏa mãn ràng buộc, False nếu có vi phạm.
    """
    I, J, K = x.shape

    for j in range(J):
        for k in range(K - 1):  # tránh truy cập ngoài mảng ở k+1
            for i in range(I):
                for i_ in range(I):
                    lhs = x[i][j][k] * TA[i]
                    rhs = x[i_][j][k+1] * TA[i_]
                    if lhs > rhs + 1e-6:  # thêm ngưỡng epsilon nhỏ để tránh sai số số học
                        print(f"Vi phạm ràng buộc tại i={i}, i'={i_}, j={j}, k={k}: {lhs:.2f} > {rhs:.2f}")
                        return False

    return True


In [ ]:
import numpy as np

# Ma trận nhị phân x[i][j][k]
x = np.zeros((3, 1, 3), dtype=int)
x[0][0][0] = 1  # Tàu 0 phục vụ ở bến 0, thứ tự 0
x[1][0][1] = 1  # Tàu 1 phục vụ ở bến 0, thứ tự 1
x[2][0][2] = 1  # Tàu 2 phục vụ ở bến 0, thứ tự 2

# Thời gian đến của mỗi tàu
TA = np.array([1, 3, 5])

# Kiểm tra ràng buộc
valid = check_constraint_order(x, TA)
print("Thỏa mãn ràng buộc!" if valid else "Không thỏa mãn!")


In [ ]:
import numpy as np

def constraints_25_to_31(q, VC, VCm, VCM, Delta_VC, TS, TD):
    """
    Kiểm tra các ràng buộc (25)-(31) về phân bổ cần cẩu bến
    
    Parameters:
        q (numpy.ndarray): Ma trận nhị phân (V x C x T), q[i][n][t] = 1 nếu cần cẩu n phục vụ tàu i tại thời điểm t
        VC (numpy.ndarray): Ma trận (V x T), VC[i][t] = số cần cẩu phục vụ tàu i tại t
        VCm (numpy.ndarray): Mảng (V), VCm[i] = giới hạn dưới cần cẩu cho tàu i
        VCM (numpy.ndarray): Mảng (V), VCM[i] = giới hạn trên cần cẩu cho tàu i
        Delta_VC (numpy.ndarray): Ma trận (V x T), Delta_VC[i][t] = VC[i][t] - VC[i][t-1]
        TS (numpy.ndarray): Mảng (V), TS[i] = thời gian bắt đầu xếp/dỡ tàu i
        TD (numpy.ndarray): Mảng (V), TD[i] = thời gian kết thúc xếp/dỡ tàu i
    
    Returns:
        bool: True nếu thỏa mãn tất cả ràng buộc, False nếu không
    """
    V, C, T = q.shape
    
    # Ràng buộc (25): Mỗi cần cẩu chỉ phục vụ 1 tàu tại mỗi thời điểm
    for n in range(C):
        for t in range(T):
            total = sum(q[i][n][t] for i in range(V))
            if total > 1:
                print(f"Vi phạm (25): Cần cẩu {n} tại t={t} phục vụ {total} tàu")
                return False
    
    # Ràng buộc (26): Thời gian làm việc phải nằm trong khoảng TS-TD
    for i in range(V):
        for t in range(T):
            for n in range(C):
                if q[i][n][t] == 1 and not (TS[i] <= t <= TD[i]):
                    print(f"Vi phạm (26): Cần cẩu {n} làm việc cho tàu {i} tại t={t} ngoài khoảng TS-TD ({TS[i]}-{TD[i]})")
                    return False
    
    # Ràng buộc (27): Số cần cẩu phục vụ tàu i = tổng q[i][n][t]
    for i in range(V):
        for t in range(T):
            total = sum(q[i][n][t] for n in range(C))
            if total != VC[i][t]:
                print(f"Vi phạm (27): Tàu {i} tại t={t} có {total} cần cẩu (yêu cầu: {VC[i][t]})")
                return False
    
    # Ràng buộc (28): Giới hạn trên/dưới cần cẩu
    for i in range(V):
        total = sum(q[i][n][t] for n in range(C) for t in range(T))
        if not (VCm[i] <= total <= VCM[i]):
            print(f"Vi phạm (28): Tàu {i} có {total} cần cẩu (yêu cầu: [{VCm[i]}, {VCM[i]}])")
            return False
    
    # Ràng buộc (29): Tổng cần cẩu ≤ tổng có sẵn
    for t in range(T):
        total = sum(q[i][n][t] for i in range(V) for n in range(C))
        if total > C:
            print(f"Vi phạm (29): Tại t={t} sử dụng {total}/{C} cần cẩu")
            return False
        
    # Ràng buộc (30): Tính Delta_VC
    for i in range(V):
        for t in range(1, T):
            if Delta_VC[i][t] != VC[i][t] - VC[i][t-1]:
                print(f"Vi phạm (30): Delta_VC[{i}][{t}] không khớp")
                return False
    
    # Ràng buộc (31): Cần cẩu không nhảy qua nhau
    for i in range(V):
        for n in range(1, C-1):
            for t in range(T):
                condition = q[i][n-1][t] + q[i][n+1][t] - q[i][n][t]
                if condition not in [-1, 0, 1]:
                    print(f"Vi phạm (31): Tàu {i}, cần cẩu {n} tại t={t} vi phạm điều kiện không nhảy qua")
                    return False
    
    return True

# Test case
V, C, T = 2, 3, 5  # 2 tàu, 3 cần cẩu, 5 thời điểm
q = np.zeros((V, C, T), dtype=int)

# Thiết lập dữ liệu mẫu
q[0][0][1:4] = 1  # Cần cẩu 0 phục vụ tàu 0 từ t=1-3
q[0][1][2:4] = 1  # Cần cẩu 1 phục vụ tàu 0 từ t=2-3
q[1][2][3:5] = 1  # Cần cẩu 2 phục vụ tàu 1 từ t=3-4

VC = np.zeros((V, T), dtype=int)
for i in range(V):
    for t in range(T):
        VC[i][t] = sum(q[i][n][t] for n in range(C))

Delta_VC = np.zeros((V, T), dtype=int)
for i in range(V):
    for t in range(1, T):
        Delta_VC[i][t] = VC[i][t] - VC[i][t-1]

result = constraints_25_to_31(
    q=q,
    VC=VC,
    VCm=np.array([1, 1]),  # Mỗi tàu cần ít nhất 1 cần cẩu
    VCM=np.array([2, 2]),  # Mỗi tàu tối đa 2 cần cẩu
    Delta_VC=Delta_VC,
    TS=np.array([1, 3]),   # Tàu 0 bắt đầu lúc t=1, tàu 1 lúc t=3
    TD=np.array([3, 5])    # Tàu 0 kết thúc lúc t=3, tàu 1 lúc t=5
)

print("Thỏa mãn tất cả ràng buộc (25)-(31)!" if result else "Có vi phạm!")

In [ ]:
import numpy as np
from scipy.stats import norm

def check_truck_constraints(CK, M, D, mu_vh, sigma_vh, mu_vl, sigma_vl, CF0, CE0):
    """
    Kiểm tra ràng buộc (32)-(37) về xe tải container.
    
    Parameters:
        CK (numpy.ndarray): Ma trận (C x T) - Số xe tải cho cần cẩu n tại thời điểm t.
        M (int): Tổng số xe tải tối đa.
        D (numpy.ndarray): Ma trận (V x B) - Khoảng cách từ bến j đến bãi (tàu i).
        mu_vh, sigma_vh (float): Tham số phân phối tốc độ không tải.
        mu_vl, sigma_vl (float): Tham số phân phối tốc độ có tải.
        CF0 (float): Hiệu suất lý thuyết của xe tải.
        CE0 (float): Hiệu suất lý thuyết của cần cẩu.
    
    Returns:
        bool: True nếu thỏa mãn tất cả ràng buộc.
    """
    C, T = CK.shape
    V, B = D.shape
    
    # Ràng buộc (32)
    for t in range(T):
        total_trucks = np.sum(CK[:, t])
        if total_trucks > M:
            print(f"Vi phạm (32): Tại t={t}, sử dụng {total_trucks}/{M} xe tải")
            return False
    
    # Ràng buộc (33)-(34) - Tính thời gian vận chuyển
    v_h = norm.rvs(mu_vh, sigma_vh)  # Lấy mẫu tốc độ không tải
    v_l = norm.rvs(mu_vl, sigma_vl)  # Lấy mẫu tốc độ có tải
    
    TH = D / v_h  # Ma trận (V x B)
    TL = D / v_l  # Ma trận (V x B)
    
    # Ràng buộc (37) - Hiệu suất cần cẩu
    for n in range(C):
        for t in range(T):
            # Giả sử mỗi cần cẩn chỉ phục vụ 1 tàu tại 1 thời điểm
            i, j = get_vessel_berth_assignment(n, t)  # Hàm giả định
            denominator = max(TH[i, j] + TL[i, j] + 1/CF0, CK[n, t]/CE0)
            CE_n = CK[n, t] / denominator
            
            if CE_n > CE0:
                print(f"Vi phạm (37): Cần cẩu {n} tại t={t} có hiệu suất vượt ngưỡng ({CE_n:.2f} > {CE0})")
                return False
    
    return True

# Test case
C, T = 3, 5  # 3 cần cẩu, 5 thời điểm
V, B = 2, 2  # 2 tàu, 2 bến
CK = np.random.randint(0, 5, size=(C, T))  # Số xe tải ngẫu nhiên
M = 10  # Tổng xe tải tối đa
D = np.array([[5, 3], [7, 4]])  # Khoảng cách (V x B)

result = check_truck_constraints(
    CK=CK,
    M=M,
    D=D,
    mu_vh=30, sigma_vh=2,  # Tốc độ không tải ~ N(30, 4)
    mu_vl=20, sigma_vl=1.5,  # Tốc độ có tải ~ N(20, 2.25)
    CF0=10,  # Hiệu suất xe tải
    CE0=50   # Hiệu suất cần cẩu
)
print("Thỏa mãn ràng buộc (32)-(37)" if result else "Có vi phạm!")

In [ ]:
import numpy as np

def check_binary_constraints(x, q, mu, nu):
    """
    Kiểm tra ràng buộc (38)-(41) về biến nhị phân.
    
    Parameters:
        x (numpy.ndarray): Ma trận 3D (V x B x VO) - Phân bổ tàu-bến-thứ tự.
        q (numpy.ndarray): Ma trận 2D (V x C) - Phân bổ cần cẩu-tàu.
        mu (numpy.ndarray): Ma trận 2D (V x U) - Biến hỗ trợ logic 1.
        nu (numpy.ndarray): Ma trận 2D (V x U) - Biến hỗ trợ logic 2.
    
    Returns:
        bool: True nếu tất cả biến là nhị phân, False nếu vi phạm.
    """
    # Kiểm tra ràng buộc (38)
    if not np.all((x == 0) | (x == 1)):
        print("Vi phạm (38): Biến x không phải nhị phân")
        return False
    
    # Kiểm tra ràng buộc (39)
    if not np.all((q == 0) | (q == 1)):
        print("Vi phạm (39): Biến q không phải nhị phân")
        return False
    
    # Kiểm tra ràng buộc (40)-(41)
    if not np.all((mu == 0) | (mu == 1)) or not np.all((nu == 0) | (nu == 1)):
        print("Vi phạm (40)-(41): Biến mu/nu không phải nhị phân")
        return False
    
    return True

# Test case
V, B, VO = 2, 3, 2  # 2 tàu, 3 bến, 2 thứ tự
C, U = 4, 2         # 4 cần cẩu, 2 dịch vụ

# Tạo dữ liệu mẫu
x = np.random.randint(0, 2, size=(V, B, VO))  # Ma trận ngẫu nhiên 0/1
q = np.random.randint(0, 2, size=(V, C))
mu = np.random.randint(0, 2, size=(V, U))
nu = np.random.randint(0, 2, size=(V, U))

# Đảm bảo tính nhị phân bằng cách gán giá trị 0/1
x = x.clip(0, 1)
q = q.clip(0, 1)
mu = mu.clip(0, 1)
nu = nu.clip(0, 1)

# Kiểm tra
result = check_binary_constraints(x, q, mu, nu)
print("Thỏa mãn ràng buộc (38)-(41)" if result else "Có vi phạm!")

In [ ]:
# Hàm tính toán
def calculate_F1(TF, TA):
    v = len(V)
    total = sum(TFi - TAi for TFi, TAi in zip(TF, TA))
    F1 = (1 / v) * total
    return F1

def calculate_pvl(speed, load):
    a = 0.02
    b = -1.67
    c = 0.46
    d = 0.03
    e = 51.17
    return a * speed * speed + b * speed + c * load + d * speed * load + e

def calculate_F2(E1, speed, lmin, lmax, Dl, Dk, n_, tn_):
    pl = calculate_pvl(speed, lmin)
    pk = calculate_pvl(speed, lmax)
    F2 = E1 * (pl * Dl + pk * Dk + n_ * tn_)
    return F2
    
def calculate_F3(TD, TS, E2, delVCit, lambda1, lambda2):
    part1 = 0
    part2 = 0
    part3 = 0
    for i in range(len(TD)):
        part1 += (TD[i] - TS[i]) * VCit[i]  # Sử dụng VCit thay vì VC
        part2 += (TD[i] - TV[i]) * delVCit[i]
        part3 += delVCit[i]
    F3 = E2 * (lambda1 * (part1 + part2) + lambda2 * part3)
    return F3

def calculate_k123(a, b, c):
    return float(1)/a, float(1)/b, float(1)/c

# Các hàm tính toán thời gian
def calculate_TA(TO, delTAi):
    result = []
    for i in range(len(TO)):
        result.append(TO[i] + delTAi[i])
    return result

def calculate_fErlang():
    delTA = []
    for i in range(len(x)):
        fxku_ = (pow(u_TA * x[i], kTA[i] - 1) * pow(u_TA * e, -1 * u_TA * x[i]))
        delTA.append(fxku_)
    return delTA

# Hàm kiểm tra ràng buộc
def constraints_9(tb, ta):
    count = 0
    for i in range(len(V)):
        if ta[i] > tb[i]:
            count += 1
            print(f"Constraints error (9). TB[{i+1}] need to greater than with TA[{i+1}]")
    if count == 0:
        print("Passed constraints (9)")

def constraints_10(tb, tg):
    result = []
    for i in range(len(V)):
        result.append(tb[i] + tg[i])
    return result

def constraints_11(ts, tc):
    count = 0
    for i in range(len(V)):
        if ts[i] < tc[i]:  # Sửa thành < vì TS phải lớn hơn hoặc bằng TC
            count += 1
            print(f"Constraints error (11). TS[{i+1}] need to greater than or equal to TC[{i+1}]")
    if count == 0:
        print("Passed constraints (11)")

def constraints_12(ts, ta, tw):
    count = 0
    for i in range(len(V)):
        if (ts[i] - ta[i]) > tw[i]:
            count += 1
            print(f"Constraints error (12). TS[{i+1}] - TA[{i+1}] need to lower than or equal with TW[{i+1}]")
    if count == 0:
        print("Passed constraints (12)") 

def constraints_13(td, ts, ve, ce, t, vcit):
    count = 0
    for i in range(len(V)):
        # Đảm bảo không chia cho 0
        if t[i] > 0 and vcit[i] > 0:
            expected_time = ve[i]/(ce[i] * t[i] * vcit[i])
            if abs((td[i] - ts[i]) - expected_time) > 1e-6:  # Dùng epsilon để so sánh số thực
                count += 1
                print(f"Constraints error with i = {i + 1}")
    if count == 0:
        print("Passed constraints (13)")

def constraints_14(te, td):
    count = 0
    for i in range(len(V)):
        if te[i] < td[i]:  # Sửa thành < vì TE phải lớn hơn hoặc bằng TD
            count += 1
            print(f"Constraints error (14). TE[{i+1}] need to greater than or equal to TD[{i+1}]")
    if count == 0:
        print("Passed constraints (14)")

def constraints_15(tf, te, tg):
    count = 0
    for i in range(len(V)):
        if abs(tf[i] - (te[i] + tg[i])) > 1e-6:  # Dùng epsilon để so sánh số thực
            count += 1
            print(f"Constraints error with i = {i + 1}")
    if count == 0:
        print("Passed constraints (15)")

def constraints_18_19(u_iu, v_iu, Tt):
    S18 = 0
    S19 = 0
    K18 = 0
    K19 = 0
    for i in range(len(Tt)):
        for j in range(len(V)):
            S18 += u_iu[j][i]
        if S18 == 1:
            K18 = i
    for i in range(len(Tt)):
        for j in range(len(V)):
            S19 += v_iu[j][i]
        if S19 == 1:
            K19 = i
    if S18 == 1:
        print(f"Pass constraints (18), tau duoc phep di trong khung thu {K18 + 1}")
    else:
        print("Contraints (18) error")
    if S19 == 1:
        print(f"Pass constraints (19), tau duoc phep di trong khung thu {K19 + 1}")
    else:
        print("Contraints (19) error")

def constraints_20(x, BD_berths, VD_ships):
    """
    Kiểm tra ràng buộc: 
    ∑_j ∑_k (x_ijk * BD_j) ≥ VD_i với ∀ i ∈ V
    """
    for i in range(len(V)):
        total_capacity = 0
        for j in range(len(B)):
            for k in range(len(VO)):
                total_capacity += x[i][j][k] * BD_berths[j]
        if total_capacity < VD_ships[i]:
            print(f"Ràng buộc không thỏa mãn cho tàu {i+1}: Tổng = {total_capacity} < Nhu cầu = {VD_ships[i]}")
            return False
    return True

def constraints_21(x, BL_berths, VL_ships):
    """
    Kiểm tra ràng buộc:
    ∑_j ∑_k (x_ijk * BL_j) ≥ VL_i với ∀ i ∈ V
    """
    for i in range(len(V)):
        total_length = 0
        for j in range(len(B)):
            for k in range(len(VO)):
                total_length += x[i][j][k] * BL_berths[j]
        if total_length < VL_ships[i]:
            print(f"Ràng buộc không thỏa mãn cho tàu {i+1}: Tổng chiều dài bến = {total_length} < Chiều dài tàu = {VL_ships[i]}")
            return False
    return True

def constraints_22_23(x):
    """
    Kiểm tra ràng buộc:
    (22) ∑_i x_ijk ≤ 1 ∀j, k
    (23) ∑_j ∑_k x_ijk = 1 ∀i
    """
    # Ràng buộc (22)
    for j in range(len(B)):
        for k in range(len(VO)):
            total = sum(x[i][j][k] for i in range(len(V)))
            if total > 1:
                print(f"Vi phạm ràng buộc (22) tại bến j={j+1}, thứ tự k={k+1} (tổng = {total})")
                return False

    # Ràng buộc (23)
    for i in range(len(V)):
        total = sum(x[i][j][k] for j in range(len(B)) for k in range(len(VO)))
        if total != 1:
            print(f"Vi phạm ràng buộc (23) tại tàu i={i+1} (tổng = {total})")
            return False

    return True

def constraint_24(x, TA):
    """
    Kiểm tra ràng buộc thứ tự thời gian phục vụ tàu:
    x_ijk * TA_i ≤ x_i'j(k+1) * TA_i', ∀i, i' ∈ V, ∀j ∈ B, ∀k ∈ VO
    """
    for j in range(len(B)):
        for k in range(len(VO) - 1):  # tránh truy cập ngoài mảng ở k+1
            for i in range(len(V)):
                for i_ in range(len(V)):
                    lhs = x[i][j][k] * TA[i]
                    rhs = x[i_][j][k+1] * TA[i_]
                    if lhs > rhs + 1e-6 and lhs > 0 and rhs > 0:  # Thêm điều kiện để tránh so sánh khi có giá trị 0
                        print(f"Vi phạm ràng buộc tại i={i+1}, i'={i_+1}, j={j+1}, k={k+1}: {lhs:.2f} > {rhs:.2f}")
                        return False
    return True

def constraints_25_to_31(q, VC, VCm, VCM, Delta_VC, TS, TD):
    """
    Kiểm tra các ràng buộc (25)-(31) về phân bổ cần cẩu bến
    """
    # Chuyển đổi các mảng 2D thành numpy arrays
    q_np = np.array(q)
    VC_np = np.array(VC)
    VCm_np = np.array(VCm)
    VCM_np = np.array(VCM)
    Delta_VC_np = np.array(Delta_VC)
    TS_np = np.array(TS)
    TD_np = np.array(TD)
    
    # Ràng buộc (25): Mỗi cần cẩu chỉ phục vụ 1 tàu tại mỗi thời điểm
    for n in range(len(C)):
        for t in range(len(T)):
            total = sum(q[i][n][t] for i in range(len(V)))
            if total > 1:
                print(f"Vi phạm (25): Cần cẩu {n+1} tại t={t+1} phục vụ {total} tàu")
                return False
    
    # Ràng buộc (26): Thời gian làm việc phải nằm trong khoảng TS-TD
    for i in range(len(V)):
        for t in range(len(T)):
            for n in range(len(C)):
                if q[i][n][t] == 1 and not (TS[i] <= t+1 <= TD[i]):  # t+1 vì T bắt đầu từ 1
                    print(f"Vi phạm (26): Cần cẩu {n+1} làm việc cho tàu {i+1} tại t={t+1} ngoài khoảng TS-TD ({TS[i]}-{TD[i]})")
                    return False
    
    # Các ràng buộc khác tương tự...
    
    return True

def check_truck_constraints(CKnt, M, D, vh, vl, CF0, CE0):
    """
    Kiểm tra ràng buộc (32)-(37) về xe tải container.
    """
    # Chuyển đổi CKnt thành numpy array nếu cần
    CK_np = np.array(CKnt)
    
    # Ràng buộc (32): Số xe tải không vượt quá M
    for t in range(len(T)):
        total_trucks = sum(CKnt[n][t] for n in range(len(C)))
        if total_trucks > M:
            print(f"Vi phạm (32): Tại t={t+1}, sử dụng {total_trucks}/{M} xe tải")
            return False
    
    # Ràng buộc (33)-(34): Tính thời gian vận chuyển
    TH = [[D[i][j]/vh for j in range(len(B))] for i in range(len(V))]
    TL = [[D[i][j]/vl for j in range(len(B))] for i in range(len(V))]
    
    # Các ràng buộc khác tương tự...
    
    return True

def check_binary_constraints(x, q, u_iu, v_iu):
    """
    Kiểm tra ràng buộc (38)-(41) về biến nhị phân.
    """
    # Kiểm tra ràng buộc (38): x_ijk là biến nhị phân
    for i in range(len(V)):
        for j in range(len(B)):
            for k in range(len(VO)):
                if x[i][j][k] not in [0, 1]:
                    print(f"Vi phạm (38): Biến x[{i+1}][{j+1}][{k+1}] = {x[i][j][k]} không phải nhị phân")
                    return False
    
    # Kiểm tra ràng buộc (39): q_itn là biến nhị phân
    for i in range(len(V)):
        for n in range(len(C)):
            for t in range(len(T)):
                if q[i][n][t] not in [0, 1]:
                    print(f"Vi phạm (39): Biến q[{i+1}][{n+1}][{t+1}] = {q[i][n][t]} không phải nhị phân")
                    return False
    
    # Kiểm tra ràng buộc (40)-(41): μ_iu, ν_iu là biến nhị phân
    for i in range(len(V)):
        for u in range(len(u_iu[0])):
            if u_iu[i][u] not in [0, 1]:
                print(f"Vi phạm (40): Biến u_iu[{i+1}][{u+1}] = {u_iu[i][u]} không phải nhị phân")
                return False
            if v_iu[i][u] not in [0, 1]:
                print(f"Vi phạm (41): Biến v_iu[{i+1}][{u+1}] = {v_iu[i][u]} không phải nhị phân")
                return False
    
    return True

In [ ]:
# Test Case for Constraints

# Import necessary libraries
import numpy as np
import random

# 1. Generate Sample Data (consistent with defined sets and parameters)

# Assuming V, B, C, T, VO are defined as before
num_vessels = len(V)
num_berths = len(B)
num_cranes = len(C)
num_time_slots = len(T)
num_orders = len(VO)
num_tidal_windows = 10 # Example value

# Decision variables (binary)
x = np.zeros((num_vessels, num_berths, num_orders), dtype=int)
q = np.zeros((num_vessels, num_cranes, num_time_slots), dtype=int)

# Assign some sample allocations for x (ensure constraint 23 is met)
assigned_vessels = set()
for k in range(num_orders):
    for j in range(num_berths):
        # Try assigning an unassigned vessel
        available_vessels = [i for i in range(num_vessels) if i not in assigned_vessels]
        if available_vessels:
            i = random.choice(available_vessels)
            x[i, j, k] = 1
            assigned_vessels.add(i)
        # Ensure all vessels are assigned if possible
        if len(assigned_vessels) == num_vessels:
            break
    if len(assigned_vessels) == num_vessels:
        break
# Ensure constraint 23 is strictly met - assign remaining vessels if any
remaining_vessels = [i for i in range(num_vessels) if i not in assigned_vessels]
for i in remaining_vessels:
    j = random.randrange(num_berths)
    k = random.randrange(num_orders)
    # Avoid violating constraint 22
    while sum(x[v_idx, j, k] for v_idx in range(num_vessels)) >= 1:
         j = random.randrange(num_berths)
         k = random.randrange(num_orders)
    x[i, j, k] = 1


# Dependent variables (time-related) - Generate plausible times
TA = sorted([random.uniform(t, t + 5) for t in TO]) # Actual arrival slightly varied from estimate
TB = [ta + random.uniform(0.1, 1) for ta in TA] # Start sailing after arrival
TC = [tb + tg for tb, tg in zip(TB, TG0)] # Entry time = start sailing + channel time
TS = [tc + random.uniform(0.1, 2) for tc in TC] # Start service after entry

# Calculate VCit based on q (will be assigned later)
VCit_calculated = [[sum(q[i, n, t] for n in range(num_cranes)) for t in range(num_time_slots)] for i in range(num_vessels)]

# Calculate TD based on VE_ships, CE, T, VCit (using placeholder VCit for now)
TD = []
for i in range(num_vessels):
    total_service_units = 0
    for t in range(num_time_slots):
        if VCit_calculated[i][t] > 0 and CE[0] > 0: # Assuming CE is similar for all cranes for simplicity here
             total_service_units += VCit_calculated[i][t] * CE[0] # Simplified: assumes constant crane efficiency and T slot = 1 hour
    
    # Avoid division by zero
    if total_service_units > 0:
        service_duration = VE_ships[i] / total_service_units
    else:
        service_duration = 10 # Assign a default duration if no cranes assigned yet

    TD.append(TS[i] + service_duration)


# Assign sample allocations for q (ensure constraints 25, 26, 29)
cranes_in_use = [[False for _ in range(num_time_slots)] for _ in range(num_cranes)]
for i in range(num_vessels):
    # Assign cranes within the vessel's service window [TS, TD]
    start_slot = max(0, int(TS[i]) - 1) # Adjust for 0-based index
    end_slot = min(num_time_slots, int(TD[i])) # Adjust for 0-based index
    
    assigned_crane_count_total = 0
    for t_idx in range(start_slot, end_slot):
        assigned_crane_count_this_slot = 0
        # Try assigning up to VCM cranes respecting VCm and total available M
        for n in range(num_cranes):
             if assigned_crane_count_total < VCM[i] and \
                assigned_crane_count_this_slot < VCM[i] and \
                not cranes_in_use[n][t_idx] and \
                sum(q[v_idx, n, t_idx] for v_idx in range(num_vessels)) == 0: # Constraint 25 check
                 
                 # Basic check for constraint 31 (simplified)
                 can_assign = True
                 if n > 0 and n < num_cranes - 1:
                     prev_crane_used = q[i, n-1, t_idx] == 1
                     next_crane_used = q[i, n+1, t_idx] == 1
                     if prev_crane_used and next_crane_used: # Prevents assigning middle if both neighbours are assigned
                         can_assign = False
                 
                 if can_assign:
                    q[i, n, t_idx] = 1
                    cranes_in_use[n][t_idx] = True
                    assigned_crane_count_this_slot += 1
                    assigned_crane_count_total += 1 # Count total cranes used over time for this vessel

        # Ensure minimum cranes VCm are assigned if possible over the duration
        if t_idx == end_slot - 1 and assigned_crane_count_total < VCm[i]:
             # Try adding more cranes in the last slot if needed and possible
             needed = VCm[i] - assigned_crane_count_total
             for n in range(num_cranes):
                 if needed <= 0: break
                 if q[i, n, t_idx] == 0 and not cranes_in_use[n][t_idx] and \
                    sum(q[v_idx, n, t_idx] for v_idx in range(num_vessels)) == 0:
                    q[i, n, t_idx] = 1
                    cranes_in_use[n][t_idx] = True
                    needed -=1


# Recalculate VCit based on the assigned q
VCit = [[sum(q[i, n, t] for n in range(num_cranes)) for t in range(num_time_slots)] for i in range(num_vessels)]

# Recalculate TD more accurately with final q and VCit
TD = []
for i in range(num_vessels):
    total_service_units = 0
    # Assuming T represents time slot duration (e.g., 1 hour)
    time_slot_duration = 1 
    for t_idx in range(num_time_slots):
         # Use the actual crane efficiency for each crane assigned
         crane_efficiency_this_slot = sum(CE[n] for n in range(num_cranes) if q[i, n, t_idx] == 1)
         total_service_units += crane_efficiency_this_slot * time_slot_duration

    if total_service_units > 0:
        service_duration = VE_ships[i] / total_service_units
    else:
        # Handle cases where no cranes are assigned - maybe default to a large duration or error
        print(f"Warning: No cranes assigned to vessel {i+1}. Setting default service duration.")
        service_duration = 24 # Example default duration

    TD.append(TS[i] + service_duration)


# Other dependent variables
TV = [ts + random.uniform(0, 1) for ts in TS] # Adjacent crane support starts shortly after TS
TE = [td + random.uniform(0.1, 1) for td in TD] # Start sailing out after service ends
TF = [te + tg for te, tg in zip(TE, TG0)] # Finish sailing out = start out + channel time

# Calculate Delta_VC
Delta_VC = np.zeros((num_vessels, num_time_slots), dtype=int)
VCit_np = np.array(VCit) # Convert to numpy array for easier calculation
for i in range(num_vessels):
    Delta_VC[i, 1:] = VCit_np[i, 1:] - VCit_np[i, :-1]


# Truck constraints data
CKnt = np.random.randint(1, 4, size=(num_cranes, num_time_slots)) # Random truck assignment per crane/time

# Tidal variables
Tt = list(range(1, num_time_slots, W*2)) # Example: High tide every W*2 hours
u_iu = [[0 for _ in range(num_tidal_windows)] for _ in range(num_vessels)]
v_iu = [[0 for _ in range(num_tidal_windows)] for _ in range(num_vessels)]
# Assign some entry/exit windows
for i in range(num_vessels):
    entry_window = random.randrange(num_tidal_windows)
    exit_window = random.randrange(num_tidal_windows)
    u_iu[i][entry_window] = 1
    v_iu[i][exit_window] = 1
    
TW = [TW0] * num_vessels # Assuming same max waiting time for all


# 2. Call Constraint Functions

print("--- Running Constraint Checks ---")

# Time-based constraints (9-15)
print("\nChecking Time Constraints (9-15):")
constraints_9(TB, TA)
TC = constraints_10(TB, TG0) # Recalculate TC based on generated TB, TG0
constraints_11(TS, TC)
constraints_12(TS, TA, TW)
# Need crane efficiency per vessel/time for constraint 13 - using average CE for simplicity
constraints_13(TD, TS, VE_ships, [np.mean(CE)]*num_vessels, [1]*num_vessels, [np.mean(VCit[i]) if sum(VCit[i])>0 else 0 for i in range(num_vessels)])
constraints_14(TE, TD)
constraints_15(TF, TE, TG0)

# Tidal constraints (18-19)
print("\nChecking Tidal Constraints (18-19):")
# Ensure Tt is defined appropriately for your model's time steps
# Assuming Tt represents the start times of the tidal windows
tidal_windows_start_times = list(range(0, num_time_slots, W))[:num_tidal_windows] # Example Tt
constraints_18_19(u_iu, v_iu, tidal_windows_start_times) # Pass the tidal window list

# Berth and Vessel constraints (20-24)
print("\nChecking Berth/Vessel Constraints (20-24):")
constraints_20(x, BD_berths, VD_ships)
constraints_21(x, BL_berths, VL_ships)
constraints_22_23(x)
constraint_24(x, TA) # Use actual arrival time TA

# Crane allocation constraints (25-31)
print("\nChecking Crane Allocation Constraints (25-31):")
# Ensure all parameters match the function definition
# Need VC (num cranes per vessel per time), VCm, VCM, Delta_VC, TS, TD
constraints_25_to_31(q, VCit, VCm, VCM, Delta_VC.tolist(), TS, TD) # Convert Delta_VC back to list

# Truck constraints (32-37)
print("\nChecking Truck Constraints (32-37):")
# Note: check_truck_constraints in the provided code has mu_vh, sigma_vh etc. but the formula uses vh, vl.
# Assuming the function should use the defined vh, vl.
# Also, the sample function assumes a get_vessel_berth_assignment function which is not defined.
# We will only check constraint 32 here as others require more setup/assumptions.
print("Checking Constraint 32 (Total Trucks):")
truck_check_32_passed = True
for t in range(num_time_slots):
    total_trucks_t = sum(CKnt[n, t] for n in range(num_cranes))
    if total_trucks_t > M:
        print(f"Vi phạm (32): Tại t={t+1}, sử dụng {total_trucks_t}/{M} xe tải")
        truck_check_32_passed = False
if truck_check_32_passed:
        print("Passed constraints (32)")
# print("Skipping constraints 33-37 due to missing dependencies/assumptions in sample function.")


# Binary variable constraints (38-41)
print("\nChecking Binary Variable Constraints (38-41):")
# The function takes mu and nu which were renamed u_iu, v_iu
check_binary_constraints(x, q, u_iu, v_iu)

print("\n--- Constraint Checks Complete ---")

NameError: name 'V' is not defined